# Monitor Assignments and Update SQLite Table with Changes
In this example, the "Sidewalk Repair" assignments will be monitored. When a sidewalk as been repaired, the corresponding work order will be updated in the SQLite table to be marked as "Completed".

In [ ]:
import sqlite3
from datetime import datetime, timedelta
import time
import pandas as pd
from arcgis.gis import GIS
from arcgis.apps import workforce

### Connect to Organization and Get the Project
Connect to ArcGIS Online and get the Project with assignments.

In [ ]:
gis = GIS("https://arcgis.com", "workforce_scripts")
item = gis.content.get("1f7b42024da544f6b1e557889e858ac6")
project = workforce.Project(item)

### Connect to the SQLite Database and Review the Work Orders
Let's review what the work order table looks like.

In [ ]:
connection = sqlite3.connect("work_orders")
df = pd.read_sql_query("select * from work_orders", connection)
df

### Monitor the Project for Completed Assignments
Let's run a loop that will check for "Completed" "Sidewalk Repair" assignments. When an assignment is returned from ArcGIS Online, let's change the value of it's status in the SQLite table from "Backlog" to "Completed". This is accomplished by leveraging the "work_order_id" field to lightweight-join the SQLite table to the workforce assignments feature service. When running the following section, complete a "Sidewalk Repair" Assignment on the mobile app.

In [ ]:
processed_orders = ["-1"]
# Run in a loop (for demo only)
for i in range(0, 12):
    print("Waiting...")
    time.sleep(5)
    where_clause = f"status=3 AND assignmentType=2 AND workOrderId NOT IN ({','.join(processed_orders)})"
    print(f"Checking for updates... {where_clause}")
    assignments = project.assignments.search(where_clause)
    for assignment in assignments:
        cur = connection.cursor()
        values = ('Completed', assignment.notes, assignment.work_order_id,)
        cur.execute("update work_orders set status=?, notes=? where id=?", values)
        connection.commit()
        processed_orders.append(assignment.work_order_id)
        print("Completed Assignment Processed")

### Verify the Changes
Let's verify that the changes were actually written to the SQLite table.

In [ ]:
df = pd.read_sql_query("select * from work_orders", connection)
df